In [1]:
import torch
import torchvision
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [2]:
import os
import random
import shutil

In [ ]:
import os
import random
import shutil

source = 'wikiart/Abstract_Expressionism'
dest = 'data/train/abstract-expressionism'
files = os.listdir(source)
no_of_files = 800

for file_name in random.sample(files, no_of_files):
    shutil.move(os.path.join(source, file_name), dest)

In [3]:
def move_shuffled_files(src, dest, num_files):
    files = os.listdir(src)
    for file_name in random.sample(files, num_files):
        shutil.move(os.path.join(src, file_name), dest)
    

In [4]:
def train_test_split():
    #wiki_dirs = ['Abstract_Expressionism', 'Cubism', 'Pointillism', 'Realism']
    wiki_dirs = ['Early_Renaissance', 'High_Renaissance']
    base_src = 'wikiart/'
    base_train_dest = 'data/train/'
    base_test_dest = 'data/validation/'
    for wiki_dir in wiki_dirs:
        src = base_src + wiki_dir
        train_dest = base_train_dest + wiki_dir
        test_dest = base_test_dest + wiki_dir
        move_shuffled_files(src, train_dest, 800)
        move_shuffled_files(src, test_dest, 200)
    
    
    

In [5]:
train_test_split()

In [6]:
train_transform = transforms.Compose([
                                      transforms.Resize([256, 256]),
                                      transforms.RandomHorizontalFlip(p=0.5),
                                      transforms.ColorJitter(0.3,0.3,0.3,0.3),
                                      transforms.RandomGrayscale(0.05),
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean=(0.5,0.5,0.5), std=(0.5,0.5,0.5))
])

test_transform = transforms.Compose([
                                      transforms.Resize([256, 256]),
                                      transforms.ToTensor(),
])

In [7]:
train_data = datasets.ImageFolder(root="data/train", transform=train_transform)
train = DataLoader(train_data, batch_size = 32, shuffle = True)

In [8]:
test_data = datasets.ImageFolder(root="data/validation", transform=test_transform)
test = DataLoader(test_data, batch_size = 32, shuffle = False)

In [9]:
class CNNBlock(torch.nn.Module):
    def __init__(self, in_channels, out_channels, conv_kernel, conv_stride, conv_padding, pool_kernel, pool_stride):
        super().__init__()
        L = [
            torch.nn.BatchNorm2d(in_channels),
            torch.nn.ReLU(),
            torch.nn.Conv2d(in_channels, out_channels, conv_kernel, conv_stride, conv_padding, dilation=1, groups=1, bias=True, padding_mode='zeros', device=None, dtype=None),
            torch.nn.BatchNorm2d(out_channels),
            torch.nn.SiLU(),
            torch.nn.MaxPool2d(pool_kernel, pool_stride),
        ]
        self.network = torch.nn.Sequential(*L)
    
    def forward(self, x):
        return self.network(x)

class CNNClassifier(torch.nn.Module):
    def __init__(self):
        super().__init__()
        in_channels = 3
        out_channels = 32
        num_classes = 2
        kernel_size = 7
        stride = 1
        padding = 3
        L = [
            CNNBlock(in_channels, out_channels, kernel_size, stride, padding, 2, 2),
            torch.nn.Dropout(p=0.05),
            CNNBlock(32, 64, 3, 1, 1, 2, 2),
            torch.nn.Dropout(p=0.1),
            CNNBlock(64, 128, 3, 1, 1, 2, 2),
            torch.nn.MaxPool2d(2),
            torch.nn.Flatten(),
            torch.nn.Linear(32768, num_classes)
        ]
        self.network = torch.nn.Sequential(*L)
        self.transforms = torchvision.transforms.Compose([torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    def forward(self, x):
        x = self.transforms(x)
        return self.network(x)

In [10]:
model = CNNClassifier()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = 0.01)

In [11]:
def train_iter(epoch, net, trainDataLoader, optimizer, criterion):
    net.train()
    total = 0
    train_loss = 0
    correct = 0

    for inputs, targets in trainDataLoader:
        optimizer.zero_grad()
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        loss = criterion(outputs,targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        total += targets.size(0)
        correct += (predicted == targets).sum().item()

    avg_loss = train_loss / len(trainDataLoader)
    accuracy = 100 * correct / total
    print('Epoch: {} \tTraining Loss: {:.25f} \tTraining Accuracy: {:.4f}'.format(epoch, avg_loss, accuracy))
    return avg_loss

In [12]:
def train_model(num_epochs, net, trainDataLoader, optimizer, criterion):
  for i in range(num_epochs):
    train_iter(i, net, trainDataLoader, optimizer, criterion)

In [13]:
train_model(5, model, train, optimizer, criterion)

Epoch: 0 	Training Loss: 16.3681514775753029766747204 	Training Accuracy: 52.0625
Epoch: 1 	Training Loss: 6.8941128039360046741990118 	Training Accuracy: 51.9375
Epoch: 2 	Training Loss: 1.4425563871860505038569045 	Training Accuracy: 57.1875
Epoch: 3 	Training Loss: 1.0281836211681365522707665 	Training Accuracy: 56.3125
Epoch: 4 	Training Loss: 0.8706555724143981711549145 	Training Accuracy: 58.1875


In [16]:
train_model(5, model, train, optimizer, criterion)

Epoch: 0 	Training Loss: 0.7992703849077225086716680 	Training Accuracy: 58.6875
Epoch: 1 	Training Loss: 0.7385431069135666160008213 	Training Accuracy: 60.5000
Epoch: 2 	Training Loss: 0.7061315894126891645754540 	Training Accuracy: 61.3125
Epoch: 3 	Training Loss: 0.7055007076263427601148237 	Training Accuracy: 61.4375
Epoch: 4 	Training Loss: 0.7054569983482360528981303 	Training Accuracy: 60.7500


In [19]:
train_iter(10, model, train, optimizer, criterion)

Epoch: 10 	Training Loss: 0.6839170277118682417238915 	Training Accuracy: 62.5625


0.6839170277118682

In [22]:
train_iter(11, model, train, optimizer, criterion)

Epoch: 11 	Training Loss: 0.6830500721931457297486645 	Training Accuracy: 61.1875


0.6830500721931457

In [25]:
train_iter(12, model, train, optimizer, criterion)

Epoch: 12 	Training Loss: 0.6783057856559753240333066 	Training Accuracy: 62.9375


0.6783057856559753

In [28]:
train_iter(13, model, train, optimizer, criterion)

Epoch: 13 	Training Loss: 0.6697379803657531605054487 	Training Accuracy: 63.5625


0.6697379803657532

In [31]:
train_iter(14, model, train, optimizer, criterion)

Epoch: 14 	Training Loss: 0.6930746066570282470920006 	Training Accuracy: 61.0625


0.6930746066570282

In [32]:
train_iter(15, model, train, optimizer, criterion)

Epoch: 15 	Training Loss: 0.6620237714052200628245259 	Training Accuracy: 63.1875


0.6620237714052201

In [35]:
train_iter(16, model, train, optimizer, criterion)

Epoch: 16 	Training Loss: 0.6603459441661834183889823 	Training Accuracy: 63.6250


0.6603459441661834

In [38]:
train_iter(17, model, train, optimizer, criterion)
train_iter(18, model, train, optimizer, criterion)

Epoch: 17 	Training Loss: 0.6563734889030456987057960 	Training Accuracy: 64.0000
Epoch: 18 	Training Loss: 0.6326209688186645863083868 	Training Accuracy: 66.4375


0.6326209688186646

In [41]:
train_iter(19, model, train, optimizer, criterion)
train_iter(20, model, train, optimizer, criterion)

Epoch: 19 	Training Loss: 0.6514371329545974864672075 	Training Accuracy: 65.3750
Epoch: 20 	Training Loss: 0.6289315843582152787405448 	Training Accuracy: 67.1250


0.6289315843582153

In [46]:
train_model(5, model, train, optimizer, criterion)

Epoch: 0 	Training Loss: 0.6381478345394134832346822 	Training Accuracy: 66.3125
Epoch: 1 	Training Loss: 0.6326023900508880304371928 	Training Accuracy: 66.3125
Epoch: 2 	Training Loss: 0.6240154558420181185596221 	Training Accuracy: 66.8750
Epoch: 3 	Training Loss: 0.6093037420511245860765825 	Training Accuracy: 67.6250
Epoch: 4 	Training Loss: 0.5877425181865691961036191 	Training Accuracy: 69.7500


In [49]:
train_iter(26, model, train, optimizer, criterion)

Epoch: 26 	Training Loss: 0.6026446127891540438525908 	Training Accuracy: 70.5625


0.602644612789154

In [52]:
train_iter(27, model, train, optimizer, criterion)

Epoch: 27 	Training Loss: 0.5638117408752441050978632 	Training Accuracy: 71.0000


0.5638117408752441

In [57]:
train_iter(28, model, train, optimizer, criterion)

Epoch: 28 	Training Loss: 0.5474176043272018787888555 	Training Accuracy: 71.0000


0.5474176043272019

In [60]:
train_iter(29, model, train, optimizer, criterion)
train_iter(30, model, train, optimizer, criterion)

Epoch: 29 	Training Loss: 0.5707770889997482166577925 	Training Accuracy: 72.1250
Epoch: 30 	Training Loss: 0.5557324373722076771286993 	Training Accuracy: 73.8750


0.5557324373722077

In [63]:
train_iter(31, model, train, optimizer, criterion)

Epoch: 31 	Training Loss: 0.5217478299140929642874198 	Training Accuracy: 73.9375


0.521747829914093

In [65]:
train_iter(32, model, train, optimizer, criterion)
train_iter(33, model, train, optimizer, criterion)
train_iter(34, model, train, optimizer, criterion)

Epoch: 32 	Training Loss: 0.7369284927845001220703125 	Training Accuracy: 66.8750
Epoch: 33 	Training Loss: 0.5420547860860824851414463 	Training Accuracy: 73.1250
Epoch: 34 	Training Loss: 0.5217531585693359064137553 	Training Accuracy: 73.5625


0.5217531585693359

In [68]:
train_iter(35, model, train, optimizer, criterion)

Epoch: 35 	Training Loss: 0.5046012270450591641335336 	Training Accuracy: 76.5625


0.5046012270450592

In [70]:
train_iter(36, model, train, optimizer, criterion)

Epoch: 36 	Training Loss: 0.4975093591213226584812901 	Training Accuracy: 75.8750


0.49750935912132266

In [72]:
train_iter(37, model, train, optimizer, criterion)

Epoch: 37 	Training Loss: 0.4814358231425285294946548 	Training Accuracy: 77.4375


0.48143582314252853

In [74]:
train_iter(38, model, train, optimizer, criterion)

Epoch: 38 	Training Loss: 0.4964321482181549005652244 	Training Accuracy: 75.8750


0.4964321482181549

In [76]:
train_iter(39, model, train, optimizer, criterion)

Epoch: 39 	Training Loss: 0.4584416246414184659130342 	Training Accuracy: 78.3125


0.45844162464141847

In [78]:
train_iter(40, model, train, optimizer, criterion)

Epoch: 40 	Training Loss: 0.4750740873813629194799546 	Training Accuracy: 78.6250


0.4750740873813629

In [66]:
def eval_model(net, testDataLoader):
    net.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in testDataLoader:
            outputs = net(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += len(targets)
            correct += (predicted == targets).sum().item()
            
    print('Accuracy of the network on the set of %d test images: %d %%' % (total,
        100 * correct / total))

In [62]:
eval_model(model, test)

Accuracy of the network on the set of 400 test images: 55 %
